In [1]:
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from models.commit import Commit
from models.file import File

Added finer to sys.path


In [4]:
def process_commit(com: Commit):
    repo_path = path.join('..', 'download', 'orgs', com.org_name, com.repo_name)
    
    processed_files = set()
    
    futures = set()
    
    with ThreadPoolExecutor(max_workers=25) as executor:
        for file in com.files:
            file_key = (file, com.sha, com.repo_name, com.org_name)
            if file_key in processed_files:
                continue
            processed_files.add(file_key)
            futures.add(executor.submit(File.get_file_content, repo_path, com.sha, file))
        
        files_to_db = []
        for future in as_completed(futures):
            file_content, file_name = future.result()
            file_to_db = File(file_name, com.sha, com.repo_name, com.org_name, file_name.split('.')[-1].lower(), file_content)
            files_to_db.append(file_to_db)
        
        File.add_files_in_batches(files_to_db)


In [5]:
commits = Commit.fetch_all_commits()
parent_folder = path.join('..', 'download', 'orgs')

In [ ]:
for com in tqdm(commits, total=len(commits), desc="Processing commits"):
    if com.files:
        process_commit(com)

Processing commits:   0%|          | 0/32533 [00:00<?, ?it/s]

Processing commits:  99%|█████████▊| 32085/32533 [2:04:02<02:28,  3.01it/s]   